In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import time
import functions

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 
                'member_information.csv', 
                'labresults_training.csv', 
                'medical_training.csv', 
                'rx_training.csv',
                'medical_target.csv',
                'rx_target.csv']

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

with open(filepath+csvfilenames[5]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetallmedical = pd.DataFrame(data)
targetallmedical.columns = targetallmedical.iloc[0]
targetallmedical = targetallmedical[1:]

with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

In [ ]:
medical_wanted = allmedical[('630' >=  allmedical.DIAG1) | (allmedical.DIAG1 >= '67999')]
medical_wanted = medical_wanted[('630' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '67999')]

medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG1) | (medical_wanted.DIAG1 >= '95999')]
medical_wanted = medical_wanted[('800' >=  medical_wanted.DIAG2) | (medical_wanted.DIAG2 >= '95999')]

medical_wanted = medical_wanted[medical_wanted['DIAG1'].str.startswith('E') == False]
medical_wanted = medical_wanted[medical_wanted['DIAG2'].str.startswith('E') == False]

medical_wanted = medical_wanted[(medical_wanted['DIAG1'].str.startswith('V3') == False) & (medical_wanted['DIAG1'].str.startswith('V22') == False) & (medical_wanted['DIAG1'].str.startswith('V23') == False) & (medical_wanted['DIAG1'].str.startswith('V24') == False) & (medical_wanted['DIAG1'].str.startswith('V28') == False) & (medical_wanted['DIAG1'].str.startswith('V91') == False)]
medical_wanted = medical_wanted[(medical_wanted['DIAG2'].str.startswith('V3') == False) & (medical_wanted['DIAG2'].str.startswith('V22') == False) & (medical_wanted['DIAG2'].str.startswith('V23') == False) & (medical_wanted['DIAG2'].str.startswith('V24') == False) & (medical_wanted['DIAG2'].str.startswith('V28') == False) & (medical_wanted['DIAG2'].str.startswith('V91') == False)]



medical_wanted_t = targetallmedical[('630' >=  targetallmedical.DIAG1) | (targetallmedical.DIAG1 >= '67999')]
medical_wanted_t = medical_wanted_t[('630' >=  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 >= '67999')]

medical_wanted_t = medical_wanted_t[('800' >=  medical_wanted_t.DIAG1) | (medical_wanted_t.DIAG1 >= '95999')]
medical_wanted_t = medical_wanted_t[('800' >=  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 >= '95999')]

medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG1'].str.startswith('E') == False]
medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG2'].str.startswith('E') == False]

medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG1'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V91') == False)]
medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG2'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V91') == False)]

In [ ]:
# Don't run this
targetallmedical.columns = targetallmedical.iloc[0]
targetallmedical = targetallmedical[1:]

medical_wanted_t = targetallmedical[('630' >=  targetallmedical.DIAG1) | (targetallmedical.DIAG1 >= '67999')]
medical_wanted_t = medical_wanted_t[('630' >=  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 >= '67999')]

medical_wanted_t = medical_wanted_t[('800' >=  medical_wanted_t.DIAG1) | (medical_wanted_t.DIAG1 >= '95999')]
medical_wanted_t = medical_wanted_t[('800' >=  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 >= '95999')]

medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG1'].str.startswith('E') == False]
medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG2'].str.startswith('E') == False]

medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG1'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V91') == False)]
medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG2'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V91') == False)]
del(targetallmedical)

In [ ]:
medical_wanted.to_csv('trainingmedicalwanted.csv')
medical_wanted_t.to_csv('targetmedicalwanted.csv')

In [ ]:
medical_wanted_t.to_csv('targetmedicalwanted.csv')
medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers

In [ ]:
medical_wanted['STD_COST']=medical_wanted['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers
targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted_t['DAYS_FROM_DIAG'] = medical_wanted_t['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
#add training data
members['totalcost'] = 0.0
members['rxcost'] = 0.0
members['medicalcost'] = 0.0

dummy = rxdata.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k],'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'rxcost'] = dummy.iat[k]
    
dummy = medical_wanted.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k], 'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'medicalcost'] = dummy.iat[k]
    

#add target data
members['targetcost'] = 0.0 
members['targetrxcost'] = 0.0
members['targetmedicalcost'] = 0.0

dummy = targetrxdata.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k],'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetrxcost'] = dummy.iat[k]

    
dummy = medical_wanted_t.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k], 'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetmedicalcost'] = dummy.iat[k]

In [ ]:
#all potentially completely useless
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
#The next several blocks add counters to the member dataframe for the number of visits to different types of facilities

In [ ]:
#emergency=['23','24', '21', '41', '20', '42']
medical_wanted_emergency = medical_wanted[(medical_wanted['POS'] =='23') | 
                                          (medical_wanted['POS'] =='24') | 
                                          (medical_wanted['POS'] =='21') |
                                          (medical_wanted['POS'] =='41') |
                                          (medical_wanted['POS'] =='20') |
                                          (medical_wanted['POS'] =='42') ]
members['emervisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_emergency.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'emervisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#psychiatric=['53', '55', '51', '52', '57'] # 51 is inpatient faciliity, 55 is res. sub. abuse
medical_wanted_psych = medical_wanted[(medical_wanted['POS'] =='53') | 
                                          #(medical_wanted['POS'] =='55') |
                                          (medical_wanted['POS'] =='52') |
                                          (medical_wanted['POS'] =='57')]
members['psychvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_psych.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'psychvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#regular=['11', '01', '81','22', '60', '49', '71', '62', '17', '50', '61', '03']
medical_wanted_office = medical_wanted[(medical_wanted['POS'] =='11') | 
                                          (medical_wanted['POS'] =='01') | 
                                          (medical_wanted['POS'] =='81') |
                                          (medical_wanted['POS'] =='22') |
                                          (medical_wanted['POS'] =='60') |
                                      (medical_wanted['POS'] =='49') | 
                                          (medical_wanted['POS'] =='71') |
                                          (medical_wanted['POS'] =='62') |
                                          (medical_wanted['POS'] =='17') |
                                      (medical_wanted['POS'] =='50') | 
                                          (medical_wanted['POS'] =='61') |
                                          (medical_wanted['POS'] =='03') ]
members['officevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_office.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'officevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#elderly=['13', '31', '32', '33']
medical_wanted_elderly = medical_wanted[(medical_wanted['POS'] =='13') | 
                                          (medical_wanted['POS'] =='31') | 
                                          (medical_wanted['POS'] =='32') |
                                          (medical_wanted['POS'] =='33') ]
members['elderlyvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_elderly.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'elderlyvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#home=['12', '14', '15']
medical_wanted_home = medical_wanted[(medical_wanted['POS'] =='12') | 
                                          (medical_wanted['POS'] =='14') | 
                                          (medical_wanted['POS'] =='15')]
members['homevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_home.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'homevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#dialysis=['65']
medical_wanted_dialysis = medical_wanted[(medical_wanted['POS'] =='65')]
members['dialysisvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_dialysis.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'dialysisvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#hospice=['34']
medical_wanted_hospice = medical_wanted[(medical_wanted['POS'] =='34')]
members['hospicevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_hospice.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'hospicevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#unknown=['99', 'None']
medical_wanted_unknown = medical_wanted[(medical_wanted['POS'] =='99')|
                                        (medical_wanted['POS'] == 'None')]
members['unknownvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_unknown.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'unknownvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
members

In [ ]:
#Plot average cost for all POS codes
location_series = medical_wanted.groupby('POS')['STD_COST'].mean()

location_series.plot.bar()
plt.ylim([0,2000])
plt.show()

In [ ]:
# Add new column for region (should find a way to avoid this with groupby, but can't currently)
members['region'] = members.apply(lambda row: functions.US_region(row), axis=1)

region_series = members.groupby('region')['totalcost'].mean()

region_series.plot.bar()
plt.show()

In [ ]:
members.to_csv('members.csv')


In [ ]:
# A collection of plots:
#Note: in this situation, we're looking for people who cost around 12k and more:
members.sort_values('targetcost', ascending = 0).iloc[6000]

In [ ]:
# Also note we're looking for a little more than the upper 15% of costs:
series = members['targetcost']
series.quantile([.15, .3, .5, .7, .85, .9 , .95, 1])

In [ ]:
members

In [ ]:

colors = {'F': 'red', 'M': 'blue', 'U': 'green'}
members.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 300000) #not the actual upper limit, ignoring ncdude
#plt.xlim(0, 3000000)
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
members.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 1100000) #not the actual upper limit, ignoring ncdude
plt.xlim(000000, 3000000)
plt.show()
#it appears we can safely exclude people who cost more than 1000000 to begin with.
#Err, include.

In [ ]:
submembers = members[members['totalcost']<1000000]

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='totalcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 300000) #not the actual upper limit, ignoring ncdude
#plt.xlim(0, 3000000)
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.show()

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='hospicevisits', y = 'totalcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 600000) #actually upper limit for this subpopulation
plt.xlim(1, 50) #ignore the 0s
plt.show()

In [ ]:
submembers = submembers[submembers['hospicevisits']<15]

In [ ]:
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
#plt.ylim(0, 100000) #not the actual upper limit
plt.xlim(0,1000000)
plt.show()

In [ ]:
#can safely remove people who spent more than 400000 on prescriptions 
#(but what's going on with the people who went from 300000 to 0???)
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='rxcost', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(300000, 800000) #ignore the 0s
plt.show()

In [ ]:
submembers = submembers[submembers['rxcost']<400000]

In [ ]:
#there's a point at which high numbers of office visits imply something
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='officevisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(200, 900) #just compressing a little
plt.show()

In [ ]:
#not so much with high numbers of emergency room visits
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='emervisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(50, 300) #just compressing a little
plt.show()

In [ ]:
#maybe with psychiatric though
colors = {'Midwest':'red', 'Northeast': 'blue', 'South':'pink', 'UN':'black', 'West':'green'}
submembers.plot.scatter(x ='psychvisits', y = 'targetcost', c = members['region'].apply(lambda x: colors[x]))
plt.ylim(0, 100000) # not upper limit for this subpopulation
plt.xlim(20, 60) #just compressing a little
plt.show()